# 02_entiry-extract
- entity extraction from news description using trasnformer `Davlan/xlm-roberta-base-ner-hrl`

### install extra library

In [ ]:
!pip install transformers torch
!pip install sentencepiece

### read stored data from DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
dir_raw_id = <YOUR RAW DIR ID>
dir_process_id = <YOUR PROCESS DIR ID>

In [ ]:
datanode = DataNode()

In [ ]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='news.json'))
meta

In [ ]:
data = json.load(fp)
data

### NER (NameEntitiesRegcognition)
- extract entity from news description

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [ ]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

In [ ]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [ ]:
text = data[0]['description']
text

In [ ]:
output = nlp(text)
output

In [ ]:
post_process(output)

### Process news to spo

In [ ]:
spo = []
for news in tqdm(data):
    description = news.get('description') 
    if description == None:
        continue
    ner = nlp(description)
    processed = post_process(ner)
    for kw in processed:
        spo.append({
            'subject': news.get('title', ""),
            'predicate': kw.get('type', ""),
            'object': kw.get('text', "")
        })

In [ ]:
df = pd.DataFrame(spo)
df['predicate'] = df['predicate'].apply(lambda x: x[2:])
df

### write DataNode to DataPlatform

In [ ]:
datanode.write(df=df, directory=dir_process_id, name="spo", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='news.json')])